In [48]:
import json
from pprint import pp

In [49]:
def exclude_keys(dict: dict, keys: list):
  return {k: v for k, v in dict.items() if k not in keys}
  

In [93]:
entries = {};
all_entries = entries
with open("../output.jsonl") as f:
  for line in f:
    pair = json.loads(line)
    entries[pair[0]] = pair[1]


In [51]:
reflexive_entries = []
for word, entry_list in entries.items():
    if word.endswith('rse'):
        for entry in entry_list:
            if 'form_of' not in entry and entry not in reflexive_entries and entry['pos'] == 'verb' and len(entry['word'].split()) == 1:
                reflexive_entries.append(entry)

reflexive_lemmas = {entry['word'] for entry in reflexive_entries}
print(len(reflexive_entries))
# reflexive_entries
# pp([{"word": entry['word'], "definitions": [entry['definitions']]} for entry in reflexive_entries if not ])
# len([{"word": entry['word'], "definitions": [entry['definitions']]} for entry in reflexive_entries if not all("reflexive" in definition.get("label", "") for definition in entry['definitions'])])
# pp([{"word": entry['word'], "definitions": [entry['definitions']]} for entry in reflexive_entries if not all("reflexive" in definition.get("label", "") for definition in entry['definitions'])])

247


In [52]:
reflexive_definitions = [
  {
    "word": entry['word'],
    "definitions": list((", ".join(list(definition.values()))) for definition in entry['definitions'])
  }
  for entry in reflexive_entries
]

In [53]:
# for defin in (reflexive_definitions):
#   pp(defin)


In [54]:
has_clean_non_combined_form_reflexive_entry = []
has_problematic_non_combined_form_reflexive_entry = []
does_not_have_non_combined_lemma_entry = []
for reflexive_entry in reflexive_entries:
  non_combined_lemma = reflexive_entry.get("word", "")[:-2]
  if entries.get(non_combined_lemma):
    non_combined_entry = entries.get(non_combined_lemma)
    triplet = (
      len(non_combined_entry) == 1,
      len(non_combined_entry[0]['definitions']) == 1,
      "only used in" in non_combined_entry[0]['definitions'][0]['definition']
    )
    if all(triplet):
      # print("in first condition?")
      has_clean_non_combined_form_reflexive_entry.append((reflexive_entry, triplet))
    else:
      # print("in else?", non_combined_entry, non_combined_entry[0]['definitions'][0])
      has_problematic_non_combined_form_reflexive_entry.append((reflexive_entry, non_combined_entry, triplet))
  else:
    does_not_have_non_combined_lemma_entry.append(reflexive_entry)


# print(len([entry for entry in [(entry, entries.get(entry['word'][:-2]), len(entries.get(entry['word'][:-2])) == 1, len(entries.get(entry['word'][:-2])[0]['definitions']) == 1, "only used in" in entries.get(entry['word'][:-2])[0]['definitions'][0]['definition']) for entry in reflexive_entries if entries.get(entry['word'][:-2])] 
#  if any([not x for x in entry[2:]])]))
# pp(([entry for entry in [(entry, entries.get(entry['word'][:-2]), len(entries.get(entry['word'][:-2])) == 1, len(entries.get(entry['word'][:-2])[0]['definitions']) == 1, "only used in" in entries.get(entry['word'][:-2])[0]['definitions'][0]['definition']) for entry in reflexive_entries if entries.get(entry['word'][:-2])] 
#  if any([not x for x in entry[2:]])])[0])

# print(len([entry for entry in [(entry, entries.get(entry['word'][:-2]), len(entries.get(entry['word'][:-2])) == 1, len(entries.get(entry['word'][:-2])[0]['definitions']) == 1, "only used in" in entries.get(entry['word'][:-2])[0]['definitions'][0]['definition']) for entry in reflexive_entries if entries.get(entry['word'][:-2])] 
#  if not any([not x for x in entry[2:]])]))

print("clean:", len(has_clean_non_combined_form_reflexive_entry))
print("problematic:", len(has_problematic_non_combined_form_reflexive_entry))
pp([entry, has_problematic_non_combined_form_reflexive_entry])

clean: 206
problematic: 1
[{'word': 'baladronearse',
  'pos': 'verb',
  'from_forms': True,
  'form_of': 'baladronear',
  'definitions': []},
 [({'word': 'incautarse',
    'pos': 'verb',
    'forms': ['me incauté',
              'incautarme',
              'incautarte',
              'te incautarías',
              'incautaros',
              'incáutese',
              'os incautabais',
              'se incautará',
              'te incautaste',
              'se incautare',
              'se incauten',
              'te incautares',
              'se incautarían',
              'os incautareis',
              'incáutate',
              'os incautarais',
              'se incautasen',
              'incautándonos',
              'te incautes',
              'nos incautáremos',
              'se incautaba',
              'me incautaría',
              'incautándote',
              'os incautéis',
              'incautarse',
              'te incautarás',
              'se incautaran',


so, for 206 (of 247 total reflexive), the non-combined form of the lemma just points right back at the reflexive lemma, 

(and for 1 more it's slightly fussier but we could still trivially wipe out the entry for the non-combined lemma (maybe hard to identify algorithmically))

In [55]:
len(does_not_have_non_combined_lemma_entry)
# does_not_have_non_combined_lemma_entry

40

In [56]:
def definition_does_have_reflexive_label(definition):
  return "reflexive" in definition.get("label", "")
  print(all(["reflexive" in definition.get("label", "") for definition in entry['definitions']]))

def definition_does_have_reflexive_in_definition(definition):
  return "reflexive" in definition.get("definition", "")

def entry_does_have_reflexive_label_on_some_definition(entry):
  return any(definition_does_have_reflexive_label(definition) or definition_does_have_reflexive_in_definition(definition) for definition in entry['definitions'])

In [57]:
does_not_have_reflexive_label = [entry for entry in reflexive_entries if not entry_does_have_reflexive_label_on_some_definition(entry)]
print(len(does_not_have_reflexive_label))
has_reflexive_on_definition = [entry for entry in reflexive_entries if any(definition_does_have_reflexive_in_definition(definition) for definition in entry['definitions'])]
print(len(has_reflexive_on_definition))
pp(has_reflexive_on_definition)

82
1
[{'word': 'enchicharse',
  'pos': 'verb',
  'forms': ['enchichado', 'me enchicho', 'me enchiché'],
  'definitions': [{'label': 'Latin America',
                   'definition': '(reflexive of enchichar) to get drunk on '
                                 'chicha'},
                  {'label': 'Andes, Colombia, Central America',
                   'definition': 'to become angry'}]}]


In [58]:
# does_not_have_reflexive_label

In [59]:
reflexive_lemmas_with_forms_pointing_to_them = set()
form_of_points_to_reflexive = []
for word_entry in entries.values():
  for entry in word_entry:
    if entry.get("form_of") in reflexive_lemmas:
      form_of_points_to_reflexive.append(entry)
      reflexive_lemmas_with_forms_pointing_to_them.add(entry.get("form_of"))

In [60]:
print(len(form_of_points_to_reflexive))
print(len(reflexive_lemmas_with_forms_pointing_to_them))
print(reflexive_lemmas_with_forms_pointing_to_them == reflexive_lemmas)


27325
247
True


In [61]:
pp(form_of_points_to_reflexive[:10])

[{'word': 'muse',
  'pos': 'verb',
  'form_of': 'musirse',
  'definitions': [{'definition': 'only used in se muse, third-person singular '
                                 'present indicative of musirse'},
                  {'definition': 'only used in te ... muse, syntactic variant '
                                 'of músete, second-person singular imperative '
                                 'of musirse'}]},
 {'word': 'jambe',
  'pos': 'verb',
  'form_of': 'jambarse',
  'definitions': [{'definition': 'only used in me jambe, first-person singular '
                                 'present subjunctive of jambarse'},
                  {'definition': 'only used in se jambe, third-person singular '
                                 'present subjunctive of jambarse'},
                  {'definition': 'only used in se ... jambe, syntactic variant '
                                 'of jámbese, third-person singular imperative '
                                 'of jambarse'}]},
 {'word':

In [62]:
complete_reflexive_entries = [word_entry for word_entry in entries.values() if any(
  entry.get("word") in reflexive_lemmas for entry in word_entry
)]
print(len(complete_reflexive_entries))
print(all([len(entry) == 1 for entry in complete_reflexive_entries]))

247
True


so *currently* these all just have a single one per entry
but that's not guaranteed to stay that way? 

(and actually this is from kaikki as of ~2024-05)

hm


In [63]:
def multi_token_forms_contains_new_forms(entry):
  forms = entry.get("forms", [])
  if not forms:
    return False
  single_token_forms, multi_token_forms = [], []
  for form in forms:
    split = form.split()
    if len(split) > 1:
      multi_token_forms.append(form)
    else:
      single_token_forms.append(form)
  second_part_of_multi_tokens_set = {token.split()[1] for token in multi_token_forms}
  single_token_forms_set = set(single_token_forms)
  second_part_of_multi_tokens_not_in_single_tokens = {x for x in second_part_of_multi_tokens_set if x not in single_token_forms_set}
  single_token_forms_not_in_second_part_of_multi_tokens_set = {x for x in single_token_forms_set if x not in second_part_of_multi_tokens_set}
  pp(locals())


multi_token_forms_contains_new_forms(entries['autoconstruirse'][0]);


{'entry': {'word': 'autoconstruirse',
           'pos': 'verb',
           'forms': ['me autoconstruyese',
                     'nos autoconstruyamos',
                     'se autoconstruya',
                     'te autoconstruyeres',
                     'me autoconstruya',
                     'autoconstruirnos',
                     'se autoconstruían',
                     'nos autoconstruiríamos',
                     'autoconstruidos',
                     'autoconstruíos',
                     'te autoconstruías',
                     'se autoconstruye',
                     'se autoconstruyera',
                     'autoconstrúyase',
                     'autoconstruirte',
                     'te autoconstruyes',
                     'autoconstruite',
                     'se autoconstruirán',
                     'me autoconstruiré',
                     'autoconstruirse',
                     'os autoconstruyerais',
                     'autoconstruyéndonos',
            

ok so trivially they do actually contain meaningful forms in the reflexive forms

...I feel like I noticed this way back in may and convinced myself they didn't matter? idk

In [64]:
all_multi_token_verb_forms = [
  {
    "word": entry['word'],
    "multi_token_forms": [form for form in entry['forms'] if len(form.split()) > 1],
    "last_token_in_multi_token_forms": [form.split()[-1] for form in entry['forms'] if len(form.split()) > 1],
    "forms": entry['forms'],
    "forms_equals_multi_token_forms": [form for form in entry['forms'] if len(form.split()) > 1] == entry['forms']
  } for word_entry in entries.values()
  for entry in word_entry
  if entry['pos'] == 'verb' and entry.get("forms") and any(
    len(form.split()) > 1 for form in entry.get("forms")
  )
]
multi_token_verb_forms_with_single_token_lemma = [
  entry for entry in all_multi_token_verb_forms
  if len(entry['word'].split()) == 1
]

# pp([exclude_keys(entry, "forms") for entry in multi_token_verb_forms_with_single_token_lemma])
# print(len(multi_token_verb_forms_with_single_token_lemma))


multi_token_verb_forms_with_single_token_lemma_with_more_than_two_tokens_in_some_form = [
  entry for entry in multi_token_verb_forms_with_single_token_lemma
  if any(len(form.split()) > 2 for form in entry['multi_token_forms'])
]

pp([exclude_keys(entry, ["forms"]) for entry in multi_token_verb_forms_with_single_token_lemma_with_more_than_two_tokens_in_some_form])
pp(len([exclude_keys(entry, ["forms"]) for entry in multi_token_verb_forms_with_single_token_lemma_with_more_than_two_tokens_in_some_form]))

[{'word': 'haigan',
  'multi_token_forms': ['used as a joke'],
  'last_token_in_multi_token_forms': ['joke'],
  'forms_equals_multi_token_forms': True},
 {'word': 'dársela',
  'multi_token_forms': ['me la doy', 'la dado', 'me la di'],
  'last_token_in_multi_token_forms': ['doy', 'dado', 'di'],
  'forms_equals_multi_token_forms': True},
 {'word': 'arreglárselas',
  'multi_token_forms': ['me las arreglo', 'me las arreglé', 'las arreglado'],
  'last_token_in_multi_token_forms': ['arreglo', 'arreglé', 'arreglado'],
  'forms_equals_multi_token_forms': True},
 {'word': 'pelársela',
  'multi_token_forms': ['me la pelo', 'me la pelé', 'la pelado'],
  'last_token_in_multi_token_forms': ['pelo', 'pelé', 'pelado'],
  'forms_equals_multi_token_forms': True},
 {'word': 'montárselo',
  'multi_token_forms': ['lo montado', 'me lo monté', 'me lo monto'],
  'last_token_in_multi_token_forms': ['montado', 'monté', 'monto'],
  'forms_equals_multi_token_forms': True},
 {'word': 'creérselo',
  'multi_token_f

In [65]:
multi_token_verb_forms_with_single_token_lemma_with_more_than_two_tokens_in_some_form_last_token_starts_differently_from_word = [entry for entry in multi_token_verb_forms_with_single_token_lemma_with_more_than_two_tokens_in_some_form if
  any(token[0] != entry['word'][0] for token in entry['last_token_in_multi_token_forms'])
]
# I... think in spanish this is actually strong evidence, like, I don't believe the first 
print(len(multi_token_verb_forms_with_single_token_lemma_with_more_than_two_tokens_in_some_form_last_token_starts_differently_from_word))
pp(multi_token_verb_forms_with_single_token_lemma_with_more_than_two_tokens_in_some_form_last_token_starts_differently_from_word)

2
[{'word': 'haigan',
  'multi_token_forms': ['used as a joke'],
  'last_token_in_multi_token_forms': ['joke'],
  'forms': ['used as a joke'],
  'forms_equals_multi_token_forms': True},
 {'word': 'haiga',
  'multi_token_forms': ['used as a joke'],
  'last_token_in_multi_token_forms': ['joke'],
  'forms': ['used as a joke'],
  'forms_equals_multi_token_forms': True}]


ok so in fact neither of those count

there's some meaningful question re, is it a problem to accidentally include "joke" as a form, but mostly I don't really care

In [66]:
[entry for entry in all_multi_token_verb_forms if "pasándolas" in [form.split()[0] for form in entry['multi_token_forms']]]

[{'word': 'pasar las de Caín',
  'multi_token_forms': ['pasadme las de Caín',
   'pasaríais las de Caín',
   'pasáremos las de Caín',
   'pasarás las de Caín',
   'páseme las de Caín',
   'pasemos las de Caín',
   'pasémosla las de Caín',
   'pasadle las de Caín',
   'pasá las de Caín',
   'pásalos las de Caín',
   'pasándose las de Caín',
   'pasándonos las de Caín',
   'pasaseis las de Caín',
   'pásalo las de Caín',
   'pasabais las de Caín',
   'páselo las de Caín',
   'pasarían las de Caín',
   'pasémoslos las de Caín',
   'pasábamos las de Caín',
   'pasadla las de Caín',
   'pásense las de Caín',
   'pasés las de Caín',
   'pasada las de Caín',
   'pasarse las de Caín',
   'pasanos las de Caín',
   'pasalas las de Caín',
   'pasaremos las de Caín',
   'pasareis las de Caín',
   'pasadnos las de Caín',
   'pasarlos las de Caín',
   'pasaban las de Caín',
   'pasaste las de Caín',
   'pásales las de Caín',
   'pasémoslas las de Caín',
   'pasásemos las de Caín',
   'pasares las de

In [67]:
entries['pasar las de Caín']

[{'word': 'pasar las de Caín',
  'pos': 'verb',
  'forms': ['pasadme las de Caín',
   'pasaríais las de Caín',
   'pasáremos las de Caín',
   'pasarás las de Caín',
   'páseme las de Caín',
   'pasemos las de Caín',
   'pasémosla las de Caín',
   'pasadle las de Caín',
   'pasá las de Caín',
   'pásalos las de Caín',
   'pasándose las de Caín',
   'pasándonos las de Caín',
   'pasaseis las de Caín',
   'pásalo las de Caín',
   'pasabais las de Caín',
   'páselo las de Caín',
   'pasarían las de Caín',
   'pasémoslos las de Caín',
   'pasábamos las de Caín',
   'pasadla las de Caín',
   'pásense las de Caín',
   'pasés las de Caín',
   'pasada las de Caín',
   'pasarse las de Caín',
   'pasanos las de Caín',
   'pasalas las de Caín',
   'pasaremos las de Caín',
   'pasareis las de Caín',
   'pasadnos las de Caín',
   'pasarlos las de Caín',
   'pasaban las de Caín',
   'pasaste las de Caín',
   'pásales las de Caín',
   'pasémoslas las de Caín',
   'pasásemos las de Caín',
   'pasares l

In [68]:
# print(len(all_first_entries))
# print(all_first_entries)

ok wellllll oy vey
it's not that many maybe tho

In [69]:
[entry for entry in all_multi_token_verb_forms if "pasándolas" in entry.get("multi_token_forms")]

[]

In [70]:
complete_reflexive_entries = [word_entry for word_entry in entries.values() if any(
  entry.get("word") in reflexive_lemmas for entry in word_entry
)]
print(len(complete_reflexive_entries))
print(all([len(entry) == 1 for entry in complete_reflexive_entries]))

247
True


In [71]:
multi_token_form_of_not_from_reflexives = [
  entry for word_entry in entries.values()
  for entry in word_entry
  if len(entry['word'].split()) > 1 and
  entry.get("form_of") and
  entry.get("form_of") not in reflexive_lemmas
]
multi_token_verb_form_of_not_from_reflexives = [
  entry for entry in multi_token_form_of_not_from_reflexives
  if entry['pos'] == 'verb'
]

In [72]:
print(len({entry['word'] for entry in multi_token_verb_form_of_not_from_reflexives}))
print(len({entry['form_of'] for entry in multi_token_verb_form_of_not_from_reflexives}))
print(len({entry['word'] for entry in multi_token_verb_form_of_not_from_reflexives if not entry.get("from_forms")}))
print(len({entry['form_of'] for entry in multi_token_verb_form_of_not_from_reflexives if not entry.get("from_forms")}))


14252
1833
40
36


ok so in fact there are quite a few that aren't from reflexives in the current config, including a handful that aren't even `from_forms`'s

In [73]:
# [entry for entry in multi_token_verb_form_of_not_from_reflexives if not entry.get("from_forms")]

In [74]:
has_multi_token_forms_is_multi_token_lemma = []
has_multi_token_forms_is_single_token_lemma = []

for word_entry in entries.values():
    for entry in word_entry:
        if entry.get("forms"):
            if any(len(form.split()) != 1 for form in entry.get('forms')):
                if len(entry['word'].split()) != 1:
                    has_multi_token_forms_is_multi_token_lemma.append(entry)
                else:
                    has_multi_token_forms_is_single_token_lemma.append(entry)


print(len(has_multi_token_forms_is_multi_token_lemma))
print(len(has_multi_token_forms_is_single_token_lemma))

8897
594


right, so the multi-token lemmas I think I just don't care about for what I'm using forms for


In [75]:
has_multi_token_forms_is_multi_token_lemma[::100]

[{'word': 'corrida de toros',
  'pos': 'noun',
  'forms': ['corridas de toros'],
  'definitions': [{'definition': 'bullfight', 'gloss': 'bullfighting event'}],
  'gender': ['f']},
 {'word': 'cómo estás',
  'pos': 'phrase',
  'forms': ['¿cómo estás'],
  'definitions': [{'label': 'informal', 'definition': 'how are you?'}]},
 {'word': 'Su Majestad',
  'pos': 'noun',
  'forms': ['Sus Majestades'],
  'definitions': [{'label': 'formal',
    'definition': 'Your Majesty; His Majesty; Her Majesty'}],
  'gender': ['m']},
 {'word': 'perro guardián',
  'pos': 'noun',
  'forms': ['perros guardianes'],
  'definitions': [{'definition': 'watchdog'}],
  'gender': ['m']},
 {'word': 'uña de caballo',
  'pos': 'noun',
  'forms': ['uñas de caballo'],
  'definitions': [{'definition': 'coltsfoot'}],
  'gender': ['f']},
 {'word': 'ácido cloroacético',
  'pos': 'noun',
  'forms': ['ácidos cloroacéticos'],
  'definitions': [{'label': 'organic chemistry',
    'definition': 'chloroacetic acid'}],
  'gender': ['m'

In [76]:
has_multi_token_forms_is_single_token_lemma_forms_dict = [
    {
        "word": entry['word'],
        "multi_token_forms": [form for form in entry.get("forms", []) if len(form.split()) != 1]
        # "forms": entry.get("forms"),
    }
    for entry in has_multi_token_forms_is_single_token_lemma
]
has_multi_token_forms_is_single_token_lemma_has_last_token_in_multi_token_form_with_different_starting_letter = []
def multi_token_form_diff_starting_letter_predicate(form: str, word):
    return len(form.split()) != 1 and form.split()[-1][0] != word[0]
for entry in has_multi_token_forms_is_single_token_lemma:
    if any(multi_token_form_diff_starting_letter_predicate(form, entry['word']) for form in entry.get("forms")):
        has_multi_token_forms_is_single_token_lemma_has_last_token_in_multi_token_form_with_different_starting_letter.append({
            "word": entry['word'],
            "offending_forms": [form for form in entry.get("forms") if multi_token_form_diff_starting_letter_predicate(form, entry['word'])],
            "forms": entry.get("forms")
        })


In [77]:
has_multi_token_forms_is_single_token_lemma_forms_does_not_have_last_token_in_multi_token_form_with_different_starting_letter = [
    entry for entry in has_multi_token_forms_is_single_token_lemma_forms_dict if not any(multi_token_form_diff_starting_letter_predicate(form, entry['word']) for form in entry.get("multi_token_forms"))
]
len(has_multi_token_forms_is_single_token_lemma_forms_does_not_have_last_token_in_multi_token_form_with_different_starting_letter)
# has_multi_token_forms_is_single_token_lemma_forms_does_not_have_last_token_in_multi_token_form_with_different_starting_letter[::10]

489

In [78]:
entries['rojigualda']

[{'word': 'rojigualda',
  'pos': 'name',
  'forms': ['la rojigualda'],
  'form_of': 'Rojigualda',
  'from_alt_of': True,
  'definitions': [{'definition': 'Alternative letter-case form of Rojigualda'}]},
 {'word': 'rojigualda',
  'pos': 'adj',
  'form_of': 'rojigualdo',
  'definitions': [{'definition': 'feminine singular of rojigualdo'}],
  'gender': ['f']}]

so there's exceptions like this
` {'word': 'mío', 'multi_token_forms': ['of singular masculine']},` but they're uncommon

I'm basically fine with regarding those in the same category as the ~18 problematic ones below, to say, "yes there are some singular exceptions but it's fine"

In [79]:
len(has_multi_token_forms_is_single_token_lemma_has_last_token_in_multi_token_form_with_different_starting_letter)

105

ok, not that many! of course, this contains 'ir', which, oy.

my suspicion is that anyway for 'ir', the actual verb forms in the multi-token forms are registered in the dictionary *anyway* so it doesn't really matter

In [80]:
entries['ir']

[{'word': 'ir',
  'pos': 'verb',
  'forms': ['váyale',
   'iremos',
   've',
   'iros',
   'iréis',
   'idas',
   'andanos',
   'váyala',
   'me iría',
   'fuéramos',
   'idos',
   'iba',
   'se vaya',
   'vayáis',
   'yéndolos',
   'os fueseis',
   'fuisteis',
   'fueras',
   'vayámoslas',
   'idlos',
   'idnos',
   'yéndoles',
   'iríais',
   'ir',
   'idlas',
   'fuereis',
   'venos',
   'váyanse',
   'vámosles',
   'te ibas',
   'vayas',
   'vayámosle',
   'te fueras',
   'andate',
   'irlos',
   'andalas',
   'irán',
   'fuerais',
   'yéndola',
   'se iban',
   'nos vayamos',
   'nos iríamos',
   'voy',
   'yéndome',
   'irás',
   'ida',
   'yéndolas',
   'andalo',
   'fueres',
   'nos vamos',
   'id',
   'vámoslas',
   'os ibais',
   'íbamos',
   'se fueron',
   'velo',
   'fuese',
   'nos fuésemos',
   'andalos',
   'vámoos',
   'fuera',
   'os iríais',
   'vaya',
   'váyanme',
   'yéndole',
   'fueses',
   'vayámonos',
   'vayás',
   'fueren',
   'vela',
   'nos fuéremos',
   '

In [81]:
has_multi_token_forms_is_single_token_lemma_has_last_token_in_multi_token_form_with_different_starting_letter

[{'word': 'lente',
  'offending_forms': ['lente m or f same meaning'],
  'forms': ['lentes', 'lente m or f same meaning']},
 {'word': 'color',
  'offending_forms': ['color m or f same meaning'],
  'forms': ['colores', 'color m or f same meaning']},
 {'word': 'radio',
  'offending_forms': ['radio m or f same meaning'],
  'forms': ['radio m or f same meaning', 'radios']},
 {'word': 'wiki',
  'offending_forms': ['wiki m or f same meaning'],
  'forms': ['wikis', 'wiki m or f same meaning']},
 {'word': 'CA', 'offending_forms': ['CC. AA.'], 'forms': ['CC. AA.']},
 {'word': 'CA', 'offending_forms': ['CC. AA.'], 'forms': ['CC. AA.']},
 {'word': 'chance',
  'offending_forms': ['chance m or f same meaning'],
  'forms': ['chance m or f same meaning', 'chances']},
 {'word': 'basis',
  'offending_forms': ['basis m or f same meaning'],
  'forms': ['basis m or f same meaning', 'basis']},
 {'word': 'ir',
  'offending_forms': ['se vaya',
   'os fueseis',
   'te fueras',
   'nos vayamos',
   'nos vamos'

In [82]:
IGNORE_MALFORMED_MULTI_TOKEN_FORMS = {"gender-neutral", 'meaning'}
has_multi_token_forms_is_single_token_lemma_has_last_token_in_multi_token_form_with_different_starting_letter_non_trivial_exception = [entry for entry in has_multi_token_forms_is_single_token_lemma_has_last_token_in_multi_token_form_with_different_starting_letter if any(form.split()[-1] not in IGNORE_MALFORMED_MULTI_TOKEN_FORMS for form in entry.get("offending_forms"))]
len(has_multi_token_forms_is_single_token_lemma_has_last_token_in_multi_token_form_with_different_starting_letter_non_trivial_exception)

18

so in fact there's very few of them that are *actually* problematic and non-trivial to catch

In [83]:
has_multi_token_forms_is_single_token_lemma_has_last_token_in_multi_token_form_with_different_starting_letter_non_trivial_exception

[{'word': 'CA', 'offending_forms': ['CC. AA.'], 'forms': ['CC. AA.']},
 {'word': 'CA', 'offending_forms': ['CC. AA.'], 'forms': ['CC. AA.']},
 {'word': 'ir',
  'offending_forms': ['se vaya',
   'os fueseis',
   'te fueras',
   'nos vayamos',
   'nos vamos',
   'se fueron',
   'nos fuésemos',
   'nos fuéremos',
   'te fueses',
   'me voy',
   'os vayáis',
   'me vaya',
   'os fuisteis',
   'se fuera',
   'se va',
   'me fuese',
   'se fueran',
   'me fui',
   'se fueren',
   'te vayás',
   'se fuesen',
   'nos fuimos',
   'nos íbamos',
   'nos fuéramos',
   'te fuiste',
   'os vais',
   'se fuere',
   'os fuerais',
   'se van',
   'te vas',
   'te vayas',
   'te fueres',
   'me fuere',
   'me fuera',
   'se vayan',
   'se fue',
   'os fuereis',
   'se fuese'],
  'forms': ['váyale',
   'iremos',
   've',
   'iros',
   'iréis',
   'idas',
   'andanos',
   'váyala',
   'me iría',
   'fuéramos',
   'idos',
   'iba',
   'se vaya',
   'vayáis',
   'yéndolos',
   'os fueseis',
   'fuisteis',
 

In [84]:
ir = entries['ir'][0]
# ir['forms']
print([form for form in ir['forms'] if multi_token_form_diff_starting_letter_predicate(form, 'ir') and form.split()[-1] not in ir['forms']])
print([form for form in ir['forms'] if len(form.split()) != 1 and form.split()[-1] not in ir['forms']])
print([form for form in ir['forms'] if multi_token_form_diff_starting_letter_predicate(form, 'ir') ])

[]
[]
['se vaya', 'os fueseis', 'te fueras', 'nos vayamos', 'nos vamos', 'se fueron', 'nos fuésemos', 'nos fuéremos', 'te fueses', 'me voy', 'os vayáis', 'me vaya', 'os fuisteis', 'se fuera', 'se va', 'me fuese', 'se fueran', 'me fui', 'se fueren', 'te vayás', 'se fuesen', 'nos fuimos', 'nos íbamos', 'nos fuéramos', 'te fuiste', 'os vais', 'se fuere', 'os fuerais', 'se van', 'te vas', 'te vayas', 'te fueres', 'me fuere', 'me fuera', 'se vayan', 'se fue', 'os fuereis', 'se fuese']


ok so indeed, even with ir, for the multi-token forms, their last token is already in forms anyway

In [85]:
"ir" in [entry['word'] for entry in has_multi_token_forms_is_single_token_lemma_has_last_token_in_multi_token_form_with_different_starting_letter]

True

In [86]:
verb_has_multi_token_forms_is_multi_token_lemma = [entry for entry in has_multi_token_forms_is_multi_token_lemma if entry['pos'] == 'verb']
print(len(verb_has_multi_token_forms_is_multi_token_lemma))
verb_has_multi_token_forms_is_multi_token_lemma[::10]

1676


[{'word': 'dar a luz',
  'pos': 'verb',
  'forms': ['dado a luz', 'di a luz', 'doy a luz'],
  'definitions': [{'label': 'idiomatic', 'definition': 'to give birth'}]},
 {'word': 'armar un san Quintín',
  'pos': 'verb',
  'forms': ['armado un san Quintín',
   'armé un san Quintín',
   'armo un san Quintín'],
  'form_of': 'armar la de san Quintín',
  'definitions': [{'definition': 'Rare form of armar la de san Quintín.'}]},
 {'word': 'echar de menos',
  'pos': 'verb',
  'forms': ['eché de menos', 'echado de menos', 'echo de menos'],
  'definitions': [{'label': 'idiomatic',
    'definition': 'to miss; to feel the absence of'}]},
 {'word': 'hacerse humo',
  'pos': 'verb',
  'forms': ['hecho humo', 'me hice humo', 'me hago humo'],
  'definitions': [{'label': 'idiomatic',
    'definition': 'to vanish into thin air; to disappear'}]},
 {'word': 'dar ganas',
  'pos': 'verb',
  'forms': ['doy ganas', 'di ganas', 'dado ganas'],
  'definitions': [{'label': 'idiomatic',
    'definition': 'to make fe

In [87]:
len([entry for entry in has_multi_token_forms_is_multi_token_lemma if entry['pos'] != 'verb'])
[entry for entry in has_multi_token_forms_is_multi_token_lemma if entry['pos'] != 'verb'][::35]

[{'word': 'corrida de toros',
  'pos': 'noun',
  'forms': ['corridas de toros'],
  'definitions': [{'definition': 'bullfight', 'gloss': 'bullfighting event'}],
  'gender': ['f']},
 {'word': 'sex shop',
  'pos': 'noun',
  'forms': ['sex shops'],
  'definitions': [{'definition': 'sex shop'}],
  'gender': ['m']},
 {'word': 'hijo de puta',
  'pos': 'intj',
  'forms': ['¡hijo de puta!'],
  'definitions': [{'label': 'derogatory, vulgar',
    'definition': 'son of a bitch!'}]},
 {'word': 'patria chica',
  'pos': 'noun',
  'forms': ['patrias chicas'],
  'definitions': [{'definition': 'hometown, native region'}],
  'gender': ['f']},
 {'word': 'país miembro',
  'pos': 'noun',
  'forms': ['países miembros'],
  'definitions': [{'definition': 'member country'}],
  'gender': ['m']},
 {'word': 'feliz Navidad y próspero Año Nuevo',
  'pos': 'phrase',
  'forms': ['¡Feliz Navidad y próspero Año Nuevo!'],
  'definitions': [{'definition': 'Merry Christmas and a Happy New Year'}]},
 {'word': 'poco hecho',


In [88]:
entry_has_no_forms = [word_entry for word_entry in entries.values() if any(
    not entry.get("forms") for entry in word_entry
)]
entry_has_no_forms_is_not_form_of = [word_entry for word_entry in entries.values() if any(
    not entry.get("forms") and not entry.get("form_of") for entry in word_entry
)]

In [89]:
print(len(entry_has_no_forms))
print(len(entry_has_no_forms_is_not_form_of))

1038278
17858


In [90]:
entry_has_no_forms_is_not_form_of[:10]

[[{'word': 'abdominales',
   'pos': 'noun',
   'definitions': [{'label': 'sports, only in the plural',
     'definition': 'series of any abdominal exercises'}],
   'gender': ['m']},
  {'word': 'abdominales',
   'pos': 'noun',
   'form_of': 'abdominal',
   'definitions': [{'label': 'only in the plural',
     'definition': 'plural of abdominal'}],
   'gender': ['m']},
  {'word': 'abdominales',
   'pos': 'adj',
   'form_of': 'abdominal',
   'definitions': [{'definition': 'plural of abdominal'}]}],
 [{'word': 'gratis',
   'pos': 'adj',
   'definitions': [{'label': 'invariable',
     'definition': 'free, without charge'}]},
  {'word': 'gratis',
   'pos': 'adv',
   'definitions': [{'definition': 'free, without charge'}]}],
 [{'word': 'abracadabra',
   'pos': 'intj',
   'definitions': [{'definition': 'abracadabra'}]}],
 [{'word': 'KGB',
   'pos': 'name',
   'definitions': [{'definition': 'KGB', 'gloss': 'soviet KGB'}],
   'gender': ['m']}],
 [{'word': 'osteo-',
   'pos': 'prefix',
   'definit

In [94]:
multiple_potential_reflexive = []
for word, entries in all_entries.items():
    if word.endswith('rse'):
        if len(entries) != 1:
            multiple_potential_reflexive.append(entries)

print(len(multiple_potential_reflexive))

2


In [95]:
multiple_potential_reflexive

[[{'word': 'verse',
   'pos': 'verb',
   'form_of': 'ver',
   'definitions': [{'definition': 'infinitive of ver combined with se'}]},
  {'word': 'verse',
   'pos': 'verb',
   'form_of': 'versar',
   'definitions': [{'definition': 'first/third-person singular present subjunctive of versar'},
    {'definition': 'third-person singular imperative of versar'}]}],
 [{'word': 'reverse',
   'pos': 'verb',
   'form_of': 'rever',
   'definitions': [{'definition': 'infinitive of rever combined with se'}]},
  {'word': 'reverse',
   'pos': 'verb',
   'form_of': 'reversar',
   'definitions': [{'definition': 'first/third-person singular present subjunctive of reversar'},
    {'definition': 'third-person singular imperative of reversar'}]}]]